<a href="https://colab.research.google.com/github/ravillaleite/CVQKD-Reconciliation/blob/main/Constru%C3%A7%C3%A3o_QC_LDPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDPC IRREGULARES

Geração de uma matriz LDPC irregular, com o polinômio do exemplo 3.1 do livro Iterative Error Correction, da Sarah Johnson. Os parâmetros do código são os seguintes:


*   $λ(x) = 1/4 + 1/2x + 1/4x^2$ e $ρ(x) = x^3$;

*   $Λ(x) = 3/7x + 3/7x^2 + 1/7x^3$ e $P(x) = x^4$;

*   $N=21$ e um total de arestas de $9 ⋅ 1 + 9 ⋅ 2 + 3 ⋅ 3 = 36$.

> 36 arestas com 4 por linha leva a $m=9$ linhas e taxa $9/21 = 3/7$.

A matriz foi construída à mão, utilizando as técnicas de **Bit-Filling** e a técnica apresentada por **Milicevic et al.**, onde os nós de variável de peso 1 se localizam na parte final do código.

*    A ideia inicial é fazer alguns testes com essa matriz, realizando a expansão quasí-cíclica e analisando o comportamento do polinômio com a escolha de diferentes valores de b - fator de expansão quasí-cíclica.

*    Posteriormente, deve ser implementado o algoritmo para produzir a matriz-base a partir do polinômio.

In [1]:
# @title Biblitecas de Suporte
import numpy as np

## Exemplo de Aplicação -

In [2]:
Matriz_LDPC = np.array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                        [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

As linhas de código da célula a seguir são utilizadas para calcular os parâmetros principais de H e checar se eles batem com os fatores de projeto. Posteriormente, serão utilizados para avaliar se a matriz expandida conserva as distribuições de graus dos polinômios e a taxa de projeto.

In [3]:
m, N = np.shape(Matriz_LDPC)

wr = np.sum(Matriz_LDPC, 1) #pesos das linhas
wc = np.sum(Matriz_LDPC, 0) #pesos das colunas

print(f'''Distribuição das linhas: {wr}
Distribuição das colunas: {wc}
''')
graus_colunas, count_colunas = np.unique(wc, return_counts=True)
graus_linhas, count_linhas = np.unique(wr, return_counts=True)

print(f'''Graus e quantidades das colunas: {graus_colunas} {count_colunas}
Graus e quantidades das linhas: {graus_linhas} {count_linhas}
''')

total_arestas = np.count_nonzero(Matriz_LDPC)
print('Total de arestas na Matriz: ', total_arestas)

#Voltando para o polinômio de distribuição da perspectiva das arestas:
graus_colunas_arestas = np.zeros(len(graus_colunas))
graus_linhas_arestas = np.zeros(len(graus_linhas))

for i in range(0, len(graus_colunas)):
  graus_colunas_arestas[i] = (graus_colunas[i] * count_colunas[i]) / total_arestas

for i in range(0, len(graus_linhas)):
  graus_linhas_arestas[i] = (graus_linhas[i] * count_linhas[i]) / total_arestas

print(f'''Coeficientes dos polinômios:
Coluna (lambda): {graus_colunas_arestas}
Linhas (rho): {graus_linhas_arestas}
''')

Distribuição das linhas: [4 4 4 4 4 4 4 4 4]
Distribuição das colunas: [2 2 2 3 2 2 3 2 2 2 2 3 1 1 1 1 1 1 1 1 1]

Graus e quantidades das colunas: [1 2 3] [9 9 3]
Graus e quantidades das linhas: [4] [9]

Total de arestas na Matriz:  36
Coeficientes dos polinômios:
Coluna (lambda): [0.25 0.5  0.25]
Linhas (rho): [1.]



Para transformar a matriz-base em uma matriz quasí-cíclica, é preciso inicialmente definir um fator de expansão **b**, e substituir cada entrada (exceto as da parte diagonal) por valores aleatórios que indicam a quantidade de rotações a serem realizadas em cada matriz identidade. Esses serão os passos realizados nas duas próximas células.
Na parte diagonal, estão distribuídas todas as variáveis de grau 1.

In [4]:
#fator de expansão
b = 7

# a matriz deve ter de 0 a b-1 rotações nas posições antes da diagonal maior
# vamos primeiro descobrir onde começa a diagonal maior (colunas de grau 1):

pos_col_grau1 = np.argwhere(graus_colunas == 1)

print(count_colunas[pos_col_grau1])

col_grau1 = int(count_colunas[pos_col_grau1].item())
print(type(col_grau1))
#for i in range(0, N - count_colunas[count_col_grau1]):
#  print(i)

[[9]]
<class 'int'>


In [5]:
print('Matriz LDPC antes da diagonal: ')

for i in Matriz_LDPC[:,:-col_grau1]:
  print(i)

# Primeiro, substituímos as posições de valor '0' por '-1':

matriz_base = np.copy(Matriz_LDPC)
matriz_base[matriz_base == 0] = -1

print('Matriz base, passo1: \n', matriz_base)

H_parcial = np.copy(matriz_base[:, :-col_grau1])
matriz_aleatoria = np.random.randint(0, b, Matriz_LDPC[:, :-col_grau1].shape)

print('Matriz aleatória: \n', matriz_aleatoria)

H_parcial[H_parcial == 1] = matriz_aleatoria[H_parcial == 1] # matriz aleatória ???
matriz_base[:, :-col_grau1] = H_parcial

print('Matriz base, passo2: \n', matriz_base)

Matriz LDPC antes da diagonal: 
[1 1 0 1 0 0 0 0 0 0 0 0]
[1 0 1 0 1 0 0 0 0 0 0 0]
[0 1 0 0 1 0 1 0 0 0 0 0]
[0 0 1 1 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 1 0 0 0 0 1]
[0 0 0 0 0 1 0 1 0 1 0 0]
[0 0 0 0 0 0 1 0 0 0 1 1]
[0 0 0 0 0 0 0 1 1 0 1 0]
[0 0 0 0 0 0 0 0 1 1 0 1]
Matriz base, passo1: 
 [[ 1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1]
 [-1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1]
 [-1 -1  1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  1 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1]]
Matriz aleatória: 
 [[2 2 3 6 1 4 2 6 6 3 5 2]
 [2 5 4 4 5 1 0 2 4 0 1 5]
 [6 5 2 5 0 4 1 5 3 5 4 1]
 [5 3 4 6 2 1 2

É preciso agora criar a função de expansão das matrizes quasí-cíclicas. Parece estar funcionando corretamente.

In [6]:
#import numpy as np

def rotaciona_identidade(b, k):
    """Gera uma matriz identidade rotacionada k vezes"""
    I = np.eye(b, dtype=int)  # Matriz identidade b x b
    return np.roll(I, shift=-k, axis=0)  # Rotaciona as linhas da matriz

def expandir_matriz(matriz, b):
    """Transforma a matriz original na matriz expandida"""
    linhas, colunas = matriz.shape
    # Criação da matriz expandida zerada
    matriz_expandida = np.zeros((linhas * b, colunas * b), dtype=int)

    for i in range(linhas):
        for j in range(colunas):
            valor = matriz[i, j]
            bloco = None

            if valor == -1:
                bloco = np.zeros((b, b), dtype=int)  # Matriz de zeros
            elif valor == 0:
                bloco = np.eye(b, dtype=int)  # Matriz identidade sem rotação
            elif 1 <= valor < b:
                bloco = rotaciona_identidade(b, valor)  # Rotaciona a matriz identidade

            if bloco is not None:
                # Inserindo o bloco na posição correta
                matriz_expandida[i*b:(i+1)*b, j*b:(j+1)*b] = bloco

    return matriz_expandida

# Teste
#b = 3  # Tamanho da matriz identidade
#matriz = np.array([[0, 1, -1],
#                   [2, 0, 1],
#                   [-1, 2, 0]])

# Cria a matriz expandida
matriz_expandida = expandir_matriz(matriz_base, b)
print(matriz_expandida[:20, :20])

[[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


Agora, serão testados os parâmetros de distribuição da matriz expandida, para diferentes valores de **b**, afim de checar se a matriz expandida mantém as propriedades da matriz inicial:

In [8]:
#matriz_expandida = np.array(np.loadtxt('Matriz_Expandida.txt'), dtype = 'int')
m, N = np.shape(matriz_expandida)

wr = np.sum(matriz_expandida, 1) #pesos das linhas
wc = np.sum(matriz_expandida, 0) #pesos das colunas

print(f'''Distribuição das linhas: {wr}
Distribuição das colunas: {wc}
''')
graus_colunas, count_colunas = np.unique(wc, return_counts=True)
graus_linhas, count_linhas = np.unique(wr, return_counts=True)

print(f'''Graus e quantidades das colunas: {graus_colunas} {count_colunas}
Graus e quantidades das linhas: {graus_linhas} {count_linhas}
''')

total_arestas = np.count_nonzero(matriz_expandida)
print('Total de arestas na Matriz: ', total_arestas)

#Voltando para o polinômio de distribuição da perspectiva das arestas:
graus_colunas_arestas = np.zeros(len(graus_colunas))
graus_linhas_arestas = np.zeros(len(graus_linhas))

for i in range(0, len(graus_colunas)):
  graus_colunas_arestas[i] = (graus_colunas[i] * count_colunas[i]) / total_arestas

for i in range(0, len(graus_linhas)):
  graus_linhas_arestas[i] = (graus_linhas[i] * count_linhas[i]) / total_arestas

print(f'''Coeficientes dos polinômios:
Coluna (lambda): {graus_colunas_arestas}
Linhas (rho): {graus_linhas_arestas}
''')

Distribuição das linhas: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
Distribuição das colunas: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

Graus e quantidades das colunas: [1 2 3] [63 63 21]
Graus e quantidades das linhas: [4] [63]

Total de arestas na Matriz:  252
Coeficientes dos polinômios:
Coluna (lambda): [0.25 0.5  0.25]
Linhas (rho): [1.]



É possível verificar que as distribuições de graus da perspectiva das arestas e a proporção de arestas de cada grau (da perspectiva de nós) foram mantidas. Isto é um bom indicador.

A meta agora é fazer o programinha que gere a matriz de base, para que ela não precise ser gerada à mão.